In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import optbinning as opt
%matplotlib inline
import pandas as pd
from sklearn.impute import SimpleImputer
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import OneHotEncoder, StandardScaler,KBinsDiscretizer, LabelEncoder
from sklearn.impute import SimpleImputer
from scipy.stats import mannwhitneyu ,chi2_contingency, anderson, f_oneway
import statsmodels.api as sm 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import warnings
warnings.filterwarnings("ignore")

(CVXPY) Apr 02 07:32:58 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 02 07:32:58 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
data=pd.read_csv("../data/application_train_vf.csv",parse_dates=["date_mensuelle"], index_col=0)

In [3]:
credit_bureau_data=pd.read_csv("../data/cb_findings.csv", index_col=0)
data=data.merge(credit_bureau_data, left_on="SK_ID_CURR", right_on="CB_SK_ID_CURR")

In [4]:
data["date_annee"]=data["date_mensuelle"].dt.year

In [5]:
data_reference = data[data["date_annee"] < 2020]

In [6]:
data_backtest = data[data["date_annee"] == 2020]

In [7]:
mandatory_columns=[]

#### Data Quality Assessment

In [ ]:
def data_integrity_assessment(df):
    if True:
        return True
    else :
        return False

In [ ]:
if data_integrity_assessment(data_backtest) == False :
    StopIteration

#### Feature Engineering 

In [ ]:
# creation of mandatory features et application of discretisation 

#### Population Scoring

In [12]:
# Application of score card 
data_reference["Segment"] = np.random.randint(0,7,data_reference.shape[0])
data_backtest["Segment"] = np.random.randint(0,7,data_backtest.shape[0])

In [15]:
toto=data_backtest["Segment"].value_counts(normalize=True).reset_index()
tata = data_reference["Segment"].value_counts(normalize=True).reset_index()

In [34]:
titi=tata.merge(toto, how="outer", on="Segment", suffixes=("_reference","_backtest"))

In [35]:
titi

,Segment,proportion_reference,proportion_backtest
0,0,0.142573,0.147169
1,1,0.143182,0.144797
2,2,0.142110,0.141982
3,3,0.143063,0.144198
4,4,0.143249,0.141330
5,5,0.143630,0.140444
6,6,0.142192,0.140079


In [20]:
def calculate_contribution_to_is(x):
    return (x["proportion_reference"] - x["proportion_backtest"])*np.log(x["proportion_reference"] / x["proportion_backtest"])

In [36]:
titi["contribution"]=titi.apply(calculate_contribution_to_is, axis=1)

In [41]:
pd.DataFrame(titi.apply(np.sum, axis=0)).T

,Segment,proportion_reference,proportion_backtest,contribution
0,21.0,1.0,1.0,0.000302


In [44]:
titi = pd.concat([titi,pd.DataFrame(titi.apply(np.sum, axis=0)).T], axis=0, ignore_index=True)

In [45]:
titi

,Segment,proportion_reference,proportion_backtest,contribution
0,0.0,0.142573,0.147169,1.458148e-04
1,1.0,0.143182,0.144797,1.811696e-05
2,2.0,0.142110,0.141982,1.155075e-07
3,3.0,0.143063,0.144198,8.971013e-06
4,4.0,0.143249,0.141330,2.587829e-05
5,5.0,0.143630,0.140444,7.147209e-05
6,6.0,0.142192,0.140079,3.163771e-05
7,21.0,1.000000,1.000000,3.020063e-04


In [56]:
titi["Segment"][-1:] = np.nan

In [57]:
titi

,Segment,proportion_reference,proportion_backtest,contribution
0,0.0,0.142573,0.147169,1.458148e-04
1,1.0,0.143182,0.144797,1.811696e-05
2,2.0,0.142110,0.141982,1.155075e-07
3,3.0,0.143063,0.144198,8.971013e-06
4,4.0,0.143249,0.141330,2.587829e-05
5,5.0,0.143630,0.140444,7.147209e-05
6,6.0,0.142192,0.140079,3.163771e-05
7,NaN,1.000000,1.000000,3.020063e-04


In [59]:
data_reference["Note"]=np.random.randint(0,1001,data_reference.shape[0])
data_backtest["Note"]=np.random.randint(0,1001,data_backtest.shape[0])

#### System Stability Index

In [ ]:
# SSI 
def system_stability_index(reference,backtest):
    dist_ref = reference["Segment"].value_counts(normalize=True).reset_index()
    dist_back = backtest["Segment"].value_counts(normalize=True).reset_index()
    dist_all = dist_ref.merge(dist_back, how="outer", on="Segment", suffixes=("_reference","_backtest"))
    dist_all["contribution"]=dist_all.apply(calculate_contribution_to_is, axis=1)
    dist_all = pd.concat([dist_all,pd.DataFrame(dist_all.apply(np.sum, axis=0)).T], axis=0, ignore_index=True)
    dist_all["Segment"][-1:] = np.nan
    return dist_all

In [ ]:
#

In [58]:
from scipy.stats import ks_2samp

In [62]:
ks_2samp(data1=data_reference["Note"], data2=data_backtest["Note"])

KstestResult(statistic=0.0033123477128031986, pvalue=0.8539633484066944, statistic_location=511, statistic_sign=1)

In [ ]:
# KS test
def kolmogorov_smirnov_test(reference, backtest,colname):
    results = ks_2samp(data1=reference[colname], data2=backtest[colname])
    if results[1] < 0.05:
        print(f"Null hypothesis is rejected : The distributions of {colname} are not the same")
    else :
        print(f"Null hypothesis is not rejected : The distributions of {colname} are the same")

In [14]:
import numpy as np
from deap import base, creator, tools, algorithms

# Simulons une variable numérique comme exemple
np.random.seed(42)  # Pour la reproductibilité
data = np.random.randint(0, 1001, size=1000)

# Nombre de segments
n_segments = 7

# Fonction pour calculer la variance intra-segment et inter-segment
def evalSegments(individual):
    # Tri des points de coupe
    cuts = sorted(individual)
    # Ajout des limites
    segments = [-np.inf] + cuts + [np.inf]
    # Calcul des moyennes et variances intra-segment
    var_intra = 0
    means = []
    for i in range(len(segments) - 1):
        segment_data = data[(data > segments[i]) & (data <= segments[i+1])]
        if len(segment_data) > 0:
            var_intra += np.var(segment_data)
            means.append(np.mean(segment_data))
    # Calcul de la variance inter-segments
    mean_global = np.mean(data)
    var_inter = np.mean([(mean - mean_global)**2 for mean in means])
    # L'objectif est de minimiser la variance intra-segment et maximiser la variance inter-segment
    return (var_intra, -var_inter)

# Configuration de l'algorithme génétique
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, np.min(data), np.max(data))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_segments-1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalSegments)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
toolbox.register("select", tools.selNSGA2)

# Paramètres de l'algorithme
population_size = 50
number_of_generations = 100
crossover_probability = 0.7
mutation_probability = 0.2

# Initialisation de la population
pop = toolbox.population(n=population_size)

# Lancement de l'algorithme génétique
final_population = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, mutpb=mutation_probability, ngen=number_of_generations, verbose=True)

# Meilleure solution
best_ind = tools.selBest(pop, 1)[0]
print("Meilleurs points de coupe pour la segmentation :", sorted(best_ind))
print("Variance intra-segment et -Variance inter-segment :", evalSegments(best_ind))

c:\Users\jeann\Desktop\M2\Credit Risk Modelling\PD_Model_Nexialog\Challenge_Nexialog_PD\.venv_pd\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\jeann\Desktop\M2\Credit Risk Modelling\PD_Model_Nexialog\Challenge_Nexialog_PD\.venv_pd\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	39    
2  	38    
3  	29    
4  	44    
5  	41    
6  	44    
7  	39    
8  	31    
9  	37    
10 	41    
11 	42    
12 	44    
13 	30    
14 	42    
15 	40    
16 	39    
17 	40    
18 	35    
19 	35    
20 	42    
21 	40    
22 	40    
23 	32    
24 	43    
25 	35    
26 	41    
27 	33    
28 	38    
29 	37    
30 	35    
31 	46    
32 	38    
33 	39    
34 	40    
35 	38    
36 	40    
37 	46    
38 	37    
39 	40    
40 	42    
41 	39    
42 	36    
43 	33    
44 	39    
45 	38    
46 	47    
47 	32    
48 	35    
49 	39    
50 	36    
51 	45    
52 	36    
53 	43    
54 	38    
55 	31    
56 	32    
57 	34    
58 	37    
59 	35    
60 	33    
61 	38    
62 	39    
63 	36    
64 	31    
65 	38    
66 	35    
67 	40    
68 	41    
69 	27    
70 	39    
71 	30    
72 	33    
73 	43    
74 	36    
75 	36    
76 	37    
77 	34    
78 	40    
79 	39    
80 	33    
81 	40    
82 	36    
83 	33    
84 	32    
85 	38    
86 	39    
87 	37    
88 	42    
89 	38    

In [13]:
np.var(data)

84014.80395900001

In [9]:
import numpy as np
import pygad

# Définition de la variable numérique à segmenter
np.random.seed(0)
variable_numerique = np.random.randint(0, 1001, size=1000)

# Définition de la fonction de coût pour minimiser la variance intra-segment
def fitness_func(solution, solution_idx, ga_instance):
    global variable_numerique
    # Tri des données selon les segments
    sorted_idx = np.argsort(solution)
    sorted_data = variable_numerique[sorted_idx]
    
    # Calcul des moyennes des segments
    segment_means = []
    start_idx = 0
    for end_idx in np.unique(solution):
        segment_means.append(np.mean(sorted_data[start_idx:end_idx]))
        start_idx = end_idx
    
    # Calcul de la variance intra-segment
    intra_segment_variance = np.mean([(x - segment_means[i])**2 for i, x in enumerate(sorted_data)])
    
    
    return intra_segment_variance

# Définition des limites pour les gènes (les segments)
num_genes = 1000
gene_space = [{"low": 0, "high": 6}] * num_genes

# Initialisation de la population
population_size = 100
ga_instance = pygad.GA(num_generations=50,
                       num_parents_mating=50,
                       sol_per_pop=population_size,
                       num_genes=num_genes,
                       gene_space=gene_space,
                       fitness_func=fitness_func,
                       parent_selection_type="tournament",
                       crossover_type="two_points",
                       mutation_type="random",
                       mutation_percent_genes=10)

# Exécution de l'algorithme génétique
ga_instance.run()

slice indices must be integers or None or have an __index__ method
Traceback (most recent call last):
  File "c:\Users\jeann\Desktop\M2\Credit Risk Modelling\PD_Model_Nexialog\Challenge_Nexialog_PD\.venv_pd\Lib\site-packages\pygad\pygad.py", line 1708, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jeann\AppData\Local\Temp\ipykernel_25612\2022504055.py", line 19, in fitness_func
    segment_means.append(np.mean(sorted_data[start_idx:end_idx]))
                                 ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
TypeError: slice indices must be integers or None or have an __index__ method
slice indices must be integers or None or have an __index__ method
Traceback (most recent call last):
  File "c:\Users\jeann\Desktop\M2\Credit Risk Modelling\PD_Model_Nexialog\Challenge_Nexialog_PD\.venv_pd\Lib\site-packages\pygad\pygad.py", line 1914, in run
    self.last_generation_fitness = self.cal_pop_fitness()

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Meilleure solution : ", solution)
print("Valeur de fitness de la meilleure solution : ", solution_fitness)

# Tri des données selon la solution trouvée
sorted_idx = np.argsort(solution)
sorted_data = variable_numerique[sorted_idx]

# Affichage des segments
start_idx = 0
for end_idx in np.unique(solution):
    print("Segment", end_idx + 1, ":", sorted_data[start_idx:end_idx+1])
    start_idx = end_idx + 1